In [2]:
from keras.models import Sequential
import pandas as pd
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
import sys, re, glob
import numpy as np
from keras.layers import Input, Dense, Flatten

/home/nsuprotivniy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data

In [3]:
def load_data(folder, rng):
    X = []

    for i in range(rng):

        packets = list()
        max_node = 0
        max_time = 0

        for file in glob.glob("{}/output_{}/*.routes".format(folder, i)):

            handle = open(file, 'r')
            data = handle.read()
            handle.close()

            _nodes = re.split('\n\n', data);

            _nodes.pop()
            for node in _nodes:

                _strs = re.findall('(\d{1,3}(?:\.\d{1,3}){3})\s+\d{1,3}(?:\.\d{1,3}){3}\s+\d{1,3}(?:\.\d{1,3}){3}\s+\w+\s+-?\d+\.\d+\s+(\d+)', node)

                strings = list()
                for _str in _strs:
                    strings.append(dict(zip(('Destination', 'Hops'), _str)))

                header = re.findall('Node:\s+(\d+)\s+Time:\s+(\d+)', node)

                max_node = max(max_node, int(header[0][0]))
                max_time = max(max_time, int(header[0][1]))

                for _str in strings:
                    _str['Node'] = int(header[0][0])
                    _str['Time'] = int(header[0][1])
                    packets.append(_str)


        table = pd.DataFrame(packets)
        time_agg = table.groupby(["Node", "Destination"]).agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        node_agg = time_agg.groupby("Node").agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        aggregate = node_agg.agg(['min', 'max', "mean", "median", "prod", "sum", "std", "var"])
        X.append(aggregate.values.flatten())

    X = np.array(X)
    X[np.isnan(X) | np.isinf(X)] = 0
    
    return X


In [5]:
malicious = load_data("../data/malicious", 100)
normal = load_data("../data/normal", 100)

In [23]:
X = np.clip(0.999, 0.001,np.concatenate((malicious, normal)))
y = np.concatenate((np.ones((100, 1)), np.zeros((100, 1))))

# Model

In [54]:
class GAN:
    def __init__(self, optimizer, shape):
        self.OPTIMIZER = optimizer
        self.OPTIMIZER = shape
        
    def generator(self):
        model = Sequential()
        model.add(Dense(2048, input_shape=(1000,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(SHAPE, activation='sigmoid'))
        return model
    
    def generator(self):
        model = Sequential()
        model.add(Dense(10, input_shape=(SHAPE,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        return model
    
    def predict(self):
        discriminator.trainable = False
        model = Sequential()
        model.add(generator)
        model.add(discriminator)
        return model
    
    def compile_models(self):
        generator.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)
        discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMIZER, metrics=['accuracy'] )
        stacked.compile(loss='binary_crossentropy', optimizer=OPTIMIZER)
        return discriminator, generator, stacked

# Train

In [50]:
def train(discriminator, generator, stacked, X, y, epochs=200, batch = 10):
    for cnt in range(epochs):
        
        ## train discriminator
        random_index =  np.random.randint(0, len(malicious) - batch)
        X_batch = X[random_index : random_index + batch]
        y_bacth = y[random_index : random_index + batch]
        
        gen_noise = np.random.normal(0, 1, (batch,1000))
        syntetic = generator.predict(gen_noise)
        
        x_combined_batch = np.concatenate((X_batch, syntetic))
        y_combined_batch = np.concatenate((y_bacth, np.zeros((batch, 1))))
        
        d_loss = discriminator.train_on_batch(x_combined_batch, y_combined_batch)
    
        # train generator
        noise = np.random.normal(0, 1, (batch,1000))
        y_mislabled = np.ones((batch, 1))
        g_loss = stacked.train_on_batch(noise, y_mislabled)
        print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

In [56]:
gan = GAN(Adam, malicious.shape[1])
discriminator, generator, stacked = gan.compile_models()
train(discriminator, generator, stacked, X, y)

epoch: 0, [Discriminator :: d_loss: 2.420988], [ Generator :: loss: 0.019146]
epoch: 1, [Discriminator :: d_loss: 2.460607], [ Generator :: loss: 0.019940]
epoch: 2, [Discriminator :: d_loss: 2.208845], [ Generator :: loss: 0.019423]
epoch: 3, [Discriminator :: d_loss: 2.663199], [ Generator :: loss: 0.015485]
epoch: 4, [Discriminator :: d_loss: 2.610235], [ Generator :: loss: 0.032229]
epoch: 5, [Discriminator :: d_loss: 2.749278], [ Generator :: loss: 0.033572]
epoch: 6, [Discriminator :: d_loss: 2.825825], [ Generator :: loss: 0.097390]
epoch: 7, [Discriminator :: d_loss: 2.774898], [ Generator :: loss: 0.015333]
epoch: 8, [Discriminator :: d_loss: 2.765658], [ Generator :: loss: 0.222694]
epoch: 9, [Discriminator :: d_loss: 2.790789], [ Generator :: loss: 0.013204]
epoch: 10, [Discriminator :: d_loss: 2.848495], [ Generator :: loss: 0.014278]
epoch: 11, [Discriminator :: d_loss: 3.053007], [ Generator :: loss: 0.013520]
epoch: 12, [Discriminator :: d_loss: 2.519361], [ Generator ::

epoch: 104, [Discriminator :: d_loss: 3.455196], [ Generator :: loss: 0.003833]
epoch: 105, [Discriminator :: d_loss: 2.682037], [ Generator :: loss: 0.009581]
epoch: 106, [Discriminator :: d_loss: 2.454702], [ Generator :: loss: 0.014291]
epoch: 107, [Discriminator :: d_loss: 2.719894], [ Generator :: loss: 0.018094]
epoch: 108, [Discriminator :: d_loss: 3.361147], [ Generator :: loss: 0.007002]
epoch: 109, [Discriminator :: d_loss: 3.069487], [ Generator :: loss: 0.005833]
epoch: 110, [Discriminator :: d_loss: 3.225455], [ Generator :: loss: 0.006112]
epoch: 111, [Discriminator :: d_loss: 3.164014], [ Generator :: loss: 0.007885]
epoch: 112, [Discriminator :: d_loss: 3.715904], [ Generator :: loss: 0.012217]
epoch: 113, [Discriminator :: d_loss: 3.026833], [ Generator :: loss: 0.008038]
epoch: 114, [Discriminator :: d_loss: 3.093276], [ Generator :: loss: 0.013474]
epoch: 115, [Discriminator :: d_loss: 3.317064], [ Generator :: loss: 0.009925]
epoch: 116, [Discriminator :: d_loss: 3.

# Preidiction

In [13]:
predict = discriminator.predict(X)

In [19]:
np.mean(y == (predict > 0.5))

0.68

# Metrics